<h2 align="center"> EL7006 Redes Neuronales y Teorı́a de Información para el Aprendizaje</h2>
<h2 align="center"> Tarea 2: Regresor basado en ITL </h2>

In [ ]:
import numpy as np
%matplotlib notebook
import matplotlib.pylab as plt

def train_regressor(x, z, Wi, method='MSE', sigma=1.0, lr=0.01, niter=1000):    
    global a, b
    w1 = Wi[0]
    w2 = Wi[1]    
    IEP = np.zeros(shape=(niter,))    
    N = len(x)
    if method is 'MEE':
        dx = np.tile(x, (N, 1))
        dx -= dx.T
        dx2 = np.tile(x**2, (N, 1))
        dx2 -= dx2.T
    for k in xrange(0, niter):
        e = z - (w1*x + w2*np.power(x, 2.0))
        IEP[k] = np.mean(np.power(w1*x + w2*x**2 - a*x - b*x**2, 2.0))
        if method is 'MEE':
            de = np.tile(e, (N, 1))
            de -= de.T
            gradw1 = -1.0*np.sum(np.multiply(np.multiply(np.exp(-0.5*np.power(de, 2.0)/sigma**2), de), dx))
            gradw2 = -1.0*np.sum(np.multiply(np.multiply(np.exp(-0.5*np.power(de, 2.0)/sigma**2), de), dx2))
            w1 -= lr*gradw1/(np.sqrt(2.0*np.pi)*(sigma**3)*(N**2))
            w2 -= lr*gradw2/(np.sqrt(2.0*np.pi)*(sigma**3)*(N**2))
        elif method is 'MSE':
            gradw1 =  2.0*np.sum(np.multiply(e, x))
            gradw2 =  2.0*np.sum(np.multiply(e, x**2))
            w1 += lr*gradw1
            w2 += lr*gradw2
    return [w1, w2], IEP

### Generación de datos de entrenamiento

In [ ]:
a = 10.0 
b = -2.0
N = 100
x = np.linspace(-2.0, 2.0, num=N);
fx = a*x + b*x**2
nu = np.random.randn(N,)*0.1
nu[np.random.rand(N,) > 0.8] += 10.0
z = fx + nu

### Entrenamiento de regresores

In [ ]:
wi = np.random.rand(2);
# 1000 iterations of MSE
[wMSE, IEPE] = train_regressor(x, z, wi, method='MSE', lr=0.0002, niter=1000);
# 200 iterations of MSE + 800 iterations of MEE
[w_tmp, IEPE_tmp] = train_regressor(x, z, wi, method='MSE', lr=0.0002, niter=200);
[wMEE, IEPC] = train_regressor(x, z, w_tmp, method='MEE', sigma=20.0, lr=1.0, niter=800);

### Gráficas de desempeño para MSE y MEE

In [ ]:
plt.close()
fig = plt.figure(figsize=(12, 5), dpi=80)
ax = fig.add_subplot(1, 2, 1)
ax.plot(x, fx, 'g-', label='Model', linewidth=2)
ax.plot(x, z, 'k.', label='Observations ')
ax.plot(x, wMSE[0]*x + wMSE[1]*x**2, 'r--', label='MSE regressor', linewidth=2)
ax.plot(x, wMEE[0]*x + wMEE[1]*x**2, 'b--', label='MEE regressor', linewidth=2)
#ax.plot(x,w1C*x+w2C*x.^2,'b--');
plt.xlabel('x'); 
plt.ylabel('y');
plt.legend(loc='lower right')
ax = fig.add_subplot(1, 2, 2);
ax.plot(np.arange(0, len(IEPE), step=1), IEPE, 'r-', label='MSE')
ax.plot(np.arange(0, len(IEPE), step=1), np.concatenate((IEPE_tmp, IEPC)), 'b-', label='MEE')
plt.xlabel('Iteration');
plt.ylabel('E[(f(x) - g(w;x))^2]')
plt.legend()
print("Comparison between regressor weights:")
print("a = %0.4f, b = %0.4f"  % (a, b))
print("MSE: w1 = %0.4f, w2 = %0.4f"  % (wMSE[0], wMSE[1]))
print("MEE: w1 = %0.4f, w2 = %0.4f"  % (wMEE[0], wMEE[1]))

### Gráficas de la PDF del error para MSE y MEE

In [ ]:
plt.close()
from sklearn.neighbors.kde import KernelDensity

x_plot = np.linspace(-10, 10, 1000)[:, np.newaxis]
kde = KernelDensity(kernel='gaussian', bandwidth=0.2)
fig = plt.figure(figsize=(10, 5), dpi=100)
ax = fig.add_subplot(1, 1, 1)
kde.fit((fx - wMSE[0]*x - wMSE[1]*x**2)[:, np.newaxis])
ax.plot(x_plot, np.exp(kde.score_samples(x_plot)), 'r-', label='MSE')
kde.fit((fx - wMEE[0]*x - wMEE[1]*x**2)[:, np.newaxis])
ax.plot(x_plot, np.exp(kde.score_samples(x_plot)), 'b-', label='MEE')
plt.xlabel('Error')
plt.ylabel('PDF of the error');
plt.title("Comparison between reconstruction error probability density functions (pdf)")
plt.legend()